Aim: This project is aimed to make your travel experience to Las Vegas Simple by Providing all nearby locations of nightlife fun spots with help of Foursquare datasets.

Problem Statement Many people come to Las Vegas each day and experience difficulty in choosing between the best and the not so known night out places. Here I and other foursquare users would try and help them with their tips and ratings for each place and help people enjoy most of their time rather fear losing something in the fake casinos

Objectives: First of all, the geojson data for Las Vegas's neighborhood is required. 2] Then, analyzing the data using the Foursquare API. 3] Use clustering to identify each area and its category. Data Source we are going to use is from Foursquare. Foursquare is a technology company that uses location intelligence to build meaningful consumer experiences and business solutions. So We are going to build a project with the help of Foursquare location data, Foursquare API provides great amount of quality data’s about locations.(cafe, restaurant etc) Using this data will allow tourists to easily decide where to go when they are in a specific city. Using techniques such as K-means clustering, I was able to get results about common venues in city. This information can be really helpful to tourists since they can focus on what they are trying to experience most during their Travel .(Food, culture, sport etc) These techniques also provides the visualization of clustering of city. k-means clustering is a method of vector quantization, originally from signal processing, that is popular for cluster analysis in data mining. k-means clustering aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster. This results in a partitioning of the data space into Voronoi cells

Conclusion So using Foursqaure data set I have successfully created a way to know all the location information handy, information such as casinos,cafes and hotels, venues are easily accessible using data science methodology

In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
!pip install folium
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframev
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print("All libraries installed and imported ! ")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

All libraries installed and imported ! 


In [3]:
CLIENT_ID = 'SOABYOTQMCH1WE5S4VL1QOOLXYT2KVF2N1NFEUK3SBJ53YTQ' # your Foursquare ID
CLIENT_SECRET = 'VZ1CMDZSZEMKBXZVQMPV0IWGCTC5XDG1FJOVXIOUCO3V5JOD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: SOABYOTQMCH1WE5S4VL1QOOLXYT2KVF2N1NFEUK3SBJ53YTQ
CLIENT_SECRET:VZ1CMDZSZEMKBXZVQMPV0IWGCTC5XDG1FJOVXIOUCO3V5JOD


In [5]:
lat=36.1699
long=-115.1398
radius=500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query=casino&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)


venues_map = folium.Map(location=[lat,long], zoom_start=11) # generate map centred around the Conrad Hotel
venues = results['response']['venues']
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]



# tranform venues into a dataframe
dataframe = json_normalize(venues)


# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [lat, long],
    radius=10,
    color='red',
    popup='Venues',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map
venue_id = '4a431d92f964a5207ca61fe3' # ID of Fremont Hotel & Casino
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id = '45a90a24f964a52020411fe3' # ID of Golden Nugget Hotel & Casino
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
result['response']['venue']['tips']['count']
df=pd.DataFrame({'Name':dataframe_filtered['name']})
df['ID']=dataframe_filtered['id']
id_list=df['ID']
type(id_list[0])
tips=[]
for i in id_list:
    venue_id=i
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        tips.append(result['response']['venue']['rating'])
    except:
        tips.append('NA')
        

df['Tips']=tips

df['lat']=dataframe_filtered['lat']
df['long']=dataframe_filtered['lng']
nodf=df
kclusters = 3
newdf=df.drop(['ID','Tips','Name'],axis=1)
newdf
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(newdf)
LV_merged = newdf

from random import randint
randomArray = []
for i in range(0,5):
    randomArray.append(randint(0,2))
newLabels = np.append(kmeans.labels_, randomArray)
cluslabels=newLabels[5:]
LV_merged['Cluster Labels'] = cluslabels
LV_merged['Name']=nodf['Name']

LV_map_clusters = folium.Map(location = [lat,long], zoom_start = 10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []

for lat ,lng, poi, cluster in zip(LV_merged["lat"],LV_merged["long"],LV_merged['Name'],LV_merged['Cluster Labels']):
    label = folium.Popup(str(poi)+" Cluster "+str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7
    ).add_to(LV_map_clusters)

LV_map_clusters

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])
7.4
7.7
